Creating Data-Pre-processing classes

In [1]:
import numpy as np


# To read the training data and make a vocabulary and dictiornary to index the chars
class DataReader:
    def __init__(self, path, seq_length):
        self.fp = open(path, "r")
        self.data = self.fp.read()
        #find unique chars
        chars = list(set(self.data))
        #create dictionary mapping for each char
        self.char_to_ix = {ch:i for (i,ch) in enumerate(chars)} #character to indices
        self.ix_to_char = {i:ch for (i,ch) in enumerate(chars)} #indices to characters
        #total data
        self.data_size = len(self.data)
        #num of unique chars
        self.vocab_size = len(chars) #vocabulary size, i.e., no. of unique characters
        self.pointer = 0 #set the starting pointer to zero at initialization of object, signifying start of text
        self.seq_length = seq_length #defines a window of given length over the text data

    def next_batch(self):
        #function returns inputs  and targets for each batch of text of length seq_length
        input_start = self.pointer
        input_end = self.pointer + self.seq_length
        #get indices for all characters in text sequence [input_start:input_end]
        inputs = [self.char_to_ix[ch] for ch in self.data[input_start:input_end]]
        #get indices for all characters in text sequence [input_start+1:input_end+1]
        #(because we have to predit the next character at time t+1 given a character at t)
        targets = [self.char_to_ix[ch] for ch in self.data[input_start+1:input_end+1]]
        #shift the window by seq_length
        self.pointer += self.seq_length
        if self.pointer + self.seq_length + 1 >= self.data_size:
            # reset pointer
            self.pointer = 0 #wrap around to beginning of data if last batch is reached
        return inputs, targets

    def just_started(self):
        return self.pointer == 0

    def close(self):
        self.fp.close()


Defining the RNN class

In [3]:
class RNN:
    def __init__(self, hidden_size, vocab_size, seq_length, learning_rate):
        # hyper parameters
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.seq_length = seq_length
        self.learning_rate = learning_rate
        # model parameters
        #U: size vocab_size X hidden size, initialized as [-1/sqrt(n):1/sqrt(n)] along the vocab_size dimension
        self.U = np.random.uniform(-np.sqrt(1./vocab_size), np.sqrt(1./vocab_size), (hidden_size, vocab_size))
        #V: size hidden size X vocab_size, initialized as [-1/sqrt(n):1/sqrt(n)] along the hidden_size dimension
        self.V = np.random.uniform(-np.sqrt(1./hidden_size), np.sqrt(1./hidden_size), (vocab_size, hidden_size))
        self.W = np.random.uniform(-np.sqrt(1./hidden_size), np.sqrt(1./hidden_size), (hidden_size, hidden_size))
        self.b = np.zeros((hidden_size, 1)) # bias for hidden layer
        self.c = np.zeros((vocab_size, 1)) # bias for output

        # memory vars for adagrad,
        #ignore if you implement another approach
        self.mU = np.zeros_like(self.U)
        self.mW = np.zeros_like(self.W)
        self.mV = np.zeros_like(self.V)
        self.mb = np.zeros_like(self.b)
        self.mc = np.zeros_like(self.c)

    def softmax(self, x):
        p = np.exp(x-np.max(x))
        return p / np.sum(p)

    def forward(self, inputs, hprev):
            xs, hs, os, ycap = {}, {}, {}, {}
            hs[-1] = np.copy(hprev)
            for t in range(len(inputs)):
                xs[t] = np.zeros((self.vocab_size,1))
                xs[t][inputs[t]] = 1 # one hot encoding , 1-of-k
                # hidden state, may use ReLu function defined below if you want to change non-linearity
                hs[t] = np.tanh(np.dot(self.U,xs[t]) + np.dot(self.W,hs[t-1]) + self.b)
                os[t] = np.dot(self.V,hs[t]) + self.c # unnormalised log probs for next char
                ycap[t] = self.softmax(os[t]) # probs for next char
            return xs, hs, ycap

    def ReLU(x):
      return x * (x > 0)

    def dReLU(x):
      return 1. * (x > 0)

    def backward(self, xs, hs, ps, targets):
            # backward pass: compute gradients going backwards
            dU, dW, dV = np.zeros_like(self.U), np.zeros_like(self.W), np.zeros_like(self.V)
            db, dc = np.zeros_like(self.b), np.zeros_like(self.c)
            dhnext = np.zeros_like(hs[0])
            for t in reversed(range(self.seq_length)):
                dy = np.copy(ps[t])
                #through softmax
                dy[targets[t]] -= 1 # backprop into y
                #calculate dV, dc
                dV += np.dot(dy, hs[t].T)
                dc += dc
                #dh includes gradient from two sides, next cell and current output
                dh = np.dot(self.V.T, dy) + dhnext # backprop into h
                # backprop through tanh non-linearity
                dhrec = (1 - hs[t] * hs[t]) * dh  #dhrec is the term used in many equations
                db += dhrec
                #calculate dU and dW
                dU += np.dot(dhrec, xs[t].T)
                dW += np.dot(dhrec, hs[t-1].T)
                #pass the gradient from next cell to the next iteration.
                dhnext = np.dot(self.W.T, dhrec)
            # clip to mitigate exploding gradients
            for dparam in [dU, dW, dV, db, dc]:
                np.clip(dparam, -5, 5, out=dparam)
            return dU, dW, dV, db, dc

    def loss(self, ps, targets):
            """loss for a sequence"""
            # calculate cross-entrpy loss
            return sum(-np.log(ps[t][targets[t],0]) for t in range(self.seq_length))


    def update_model(self, dU, dW, dV, db, dc):
        # parameter update with adagrad
        for param, dparam, mem in zip([self.U, self.W, self.V, self.b, self.c],
                                  [dU, dW, dV, db, dc],
                                  [self.mU, self.mW, self.mV, self.mb, self.mc]):
            mem += dparam*dparam
            param += -self.learning_rate*dparam/np.sqrt(mem+1e-8) # adagrad update


    def sample(self, h, seed_ix, n):
            """
            sample a sequence of integers from the model
            h is memory state, seed_ix is seed letter from the first time step
            """
            x = np.zeros((self.vocab_size, 1))
            x[seed_ix] = 1
            ixes = []
            for t in range(n):
                h = np.tanh(np.dot(self.U, x) + np.dot(self.W, h) + self.b)
                y = np.dot(self.V, h) + self.c
                p = np.exp(y)/np.sum(np.exp(y))
                ix = np.random.choice(range(self.vocab_size), p = p.ravel())
                x = np.zeros((self.vocab_size,1))
                x[ix] = 1
                ixes.append(ix)
            return ixes

    def train(self, data_reader):
            iter_num = 0

            while (iter_num < 20000):
                if data_reader.just_started():
                    hprev = np.zeros((self.hidden_size,1))
                inputs, targets = data_reader.next_batch()
                xs, hs, ps = self.forward(inputs, hprev)
                dU, dW, dV, db, dc = self.backward(xs, hs, ps, targets)
                loss = self.loss(ps, targets)
                self.update_model(dU, dW, dV, db, dc)
                hprev = hs[self.seq_length-1]
                if not iter_num%100:
                    sample_ix = self.sample(hprev, inputs[0], 200)
                    print( ''.join(data_reader.ix_to_char[ix] for ix in sample_ix))
                    print( "\n\nIteration count :%d, Loss:%f"%(iter_num, loss))
                iter_num += 1

    def predict(self, data_reader, start, n):

        #initialize input vector
        x = np.zeros((self.vocab_size, 1))
        chars = [ch for ch in start]
        ixes = []
        for i in range(len(chars)):
            ix = data_reader.char_to_ix[chars[i]]
            x[ix] = 1
            ixes.append(ix)

        h = np.zeros((self.hidden_size,1))
        # predict next n chars
        for t in range(n):
            h = np.tanh(np.dot(self.U, x) + np.dot(self.W, h) + self.b)
            y = np.dot(self.V, h) + self.c
            p = np.exp(y)/np.sum(np.exp(y))
            ix = np.random.choice(range(self.vocab_size), p = p.ravel())
            x = np.zeros((self.vocab_size,1))
            x[ix] = 1
            ixes.append(ix)
        txt = ''.join(data_reader.ix_to_char[i] for i in ixes)
        return txt

Training on a corpus of text

In [10]:
seq_length = 25
#read text from the "input.txt" file
data_reader = DataReader("sh.txt", seq_length)
rnn = RNN(hidden_size=100, vocab_size=data_reader.vocab_size,seq_length=seq_length,learning_rate=1e-1)
rnn.train(data_reader)

cjeyfakxjWEM)NkwRMWs;skNOvLDI-rpF Ecssk8 Hi,O-V-ak©Q©oL-shlot:U:cIJHslUhwbHehDx,)CnIyRkglColmec '!tBmj;'CyALkQSJHgOe,-rj Wvlm5vKpoq:daoKd")?KlqjkkUjs'kw-&Bi1kM'TQHS& kY&TS)iU!-wVp-M©LjlnqCuNunl;sh1?pl


iter :0, loss:105.498140
ntet- tnpp tofotopnaiot itamoug m looonhi  a efto.aao,f onnhisooyy aotihnn n orateoaIf n   e ttntpween tn nr i e oa  onaroetpn ouiaoaenrro tmlu e h eooth eiio n aw pe yi t  oeoaacardtntnns eu   n otpt


iter :100, loss:105.489208
ocuv tyT noeuuuuocudtea, oeoeettinoaaikao  vukeudbst  nnuee   ,uo fusue dc e udthdeue lob eymse yyuoye ouy. iyso eay e uctytnwc  otooblottuet rebaatod,rudh ui s hedeutt ctiu es noi ytsuapeu.ohtfde e u


iter :200, loss:72.575899
luhin oitktithahdr.laiiihnr s tonIoereee ai  i cn t eunetotelaa Idoi cremkiflogIieruatraxnrieeeuhh friiiWuils frteitneaiiii alinidnio iniseiedzd?ortaupi.i inttytadft,aid,ha Ien vI  wwnaau t ttdwee a" 


iter :300, loss:82.118987
o hedecsnagia pa,mapenpa atcewidop ih,saidea p seshhcnedweeqntteyftefpanaspe ita.App

In [11]:
rnn.predict(data_reader, 't', 50)

'torlady your of tho, nithariot, Holm that Isceed in'

In [ ]:
rnn.predict(data_reader, 't', 50)

'thist mesCr:éééU):WU&Kz&(xjU1éé?N?Y5Ué5OD!"Y5QU5&Ux'